<a href="https://colab.research.google.com/github/niranju23/Apple-iPhone-brand-monitoring/blob/main/RNN%20training_testing_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [4]:
# Load your dataset
data = pd.read_excel("/content/sample_data/final_nlp_preprocess_with_ratings.xlsx")  # Replace with your data file

In [5]:
# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Lemmatized Review'])

# Get the number of unique words
num_unique_words = len(tokenizer.word_index)

print(f"Number of unique words: {num_unique_words}")

Number of unique words: 8912


In [6]:
# Tokenize text data to get sequences
tokenizer = Tokenizer(num_words=8000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['Lemmatized Review'])
sequences = tokenizer.texts_to_sequences(data['Lemmatized Review'])

# Calculate the maximum sequence length
max_sequence_length = max(len(seq) for seq in sequences)

print(f"Maximum sequence length: {max_sequence_length}")

Maximum sequence length: 1536


In [9]:
# Define constants
num_words = 8912  # Number of unique words in your vocabulary
max_sequence_length = 1536  # Maximum length of input sequences

In [10]:
# Tokenize text and create sequences
tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(data['Lemmatized Review'])
sequences = tokenizer.texts_to_sequences(data['Lemmatized Review'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')


In [11]:
# Convert ratings to sentiment labels (1 for positive, 0 for negative)
data['sentiment'] = np.where(data['rating'] >= 4, 1, 0)

In [12]:
# Create the RNN model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_sequence_length))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))  # Binary classification

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
# Split data into training and testing sets
train_size = int(0.8 * len(data))
X_train = padded_sequences[:train_size]
y_train = data['sentiment'][:train_size]
X_test = padded_sequences[train_size:]
y_test = data['sentiment'][train_size:]

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
270/270 [==============================] - 281s 1s/step - loss: 0.6929 - accuracy: 0.5115 - val_loss: 0.6983 - val_accuracy: 0.4528
Epoch 2/5
270/270 [==============================] - 274s 1s/step - loss: 0.6929 - accuracy: 0.5159 - val_loss: 0.6959 - val_accuracy: 0.4528
Epoch 3/5
270/270 [==============================] - 273s 1s/step - loss: 0.6928 - accuracy: 0.5159 - val_loss: 0.6970 - val_accuracy: 0.4528
Epoch 4/5
270/270 [==============================] - 273s 1s/step - loss: 0.6926 - accuracy: 0.5159 - val_loss: 0.6937 - val_accuracy: 0.4528
Epoch 5/5
270/270 [==============================] - 272s 1s/step - loss: 0.6927 - accuracy: 0.5142 - val_loss: 0.7008 - val_accuracy: 0.4528


In [32]:
model.save("rnn.h5")

In [53]:
# Example prediction
sample_review = ["This product is not good! I hate it."]
sample_sequence = tokenizer.texts_to_sequences(sample_review)
sample_padded_sequence = pad_sequences(sample_sequence, maxlen=max_sequence_length, padding='post', truncating='post')
prediction = model.predict(sample_padded_sequence)

# Print the prediction
sentiment = "Positive" if prediction >= 0.5 else "Negative"
print(f"Sample Review: {sample_review[0]} | Predicted Sentiment: {sentiment}")

1/1 [==============================] - 0s 434ms/step
Sample Review: This product is not good! I hate it. | Predicted Sentiment: Positive
